# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
import pandas as pd

doc = []
with open('assets/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [3]:
def date_sorter():
    import regex as re
    
    dates=df.str.extractall(r'''(?P<date>\d{1,2}[/-]\d{1,2}[/-]\d{2}(?!\d)
                |\d{1,2}[/-]\d{1,2}[/-]\d{4}
                |(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[\s.-]\s?\d{1,2}[\s,.-]\s?\d{4}
                |(?<!\d\s)(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*,\s\d{4}
                |\d{1,2}\s?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[\s,.]\s?\d{4}
                |(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s\d{1,2}\w{2},\s\d{4}                
                |(?<!\d\s)(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s\d{4}
                |^(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s\d{4}
                |(?<!/)\d{1,2}/\d{4}
                |(?<![\d-])\d{4}(?=\D)
                |^\d{4}
                |(?<!\d)\d{4}$)''',re.VERBOSE)
    dates = dates.droplevel(1)
    dates['date']=dates['date'].str.replace(r'[.,]','',regex=True)
    dates['date']=dates['date'].str.replace(r'[ /]','-',regex=True)
    dates['date']=dates['date'].str.split('-')
    month_abv = dict({'January': 'Jan',
                 'February' : 'Feb',
                 'March' : 'Mar',
                 'April' : 'Apr',
                 'June' : 'Jun',
                 'July' : 'Jul',
                 'August' : 'Aug',
                 'September' : 'Sep',
                 'October' : 'Oct',
                 'November' : 'Nov',
                 'December' : 'Dec'})
    for x in dates['date']:
        for i in range(len(x)):
            if x[i] in month_abv.keys():
                x[i]=month_abv[x[i]]
                
    def swap(lst,index1,index2):
        swap_lst = lst
        swap_lst[index1], swap_lst[index2] = swap_lst[index2], swap_lst[index1]
        return swap_lst

    month_list = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    dates['date'] = dates['date'].apply(lambda x: swap(x,0,1) if len(x)>1 and x[1] in month_list else x)
    
    lookup_month = dict({'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12})
    dates['date']=dates['date'].apply(lambda x: [1,1]+x if len(x)==1 else(swap([1]+x,0,1) if len(x)==2 else x))
    
    mdy_df = pd.DataFrame(dates['date'].to_list(), columns=['month', 'day','year'])
    mdy_df['month']=mdy_df['month'].replace('Janaury','Jan').replace('Decemeber','Dec').replace('2June','Jun').replace(lookup_month)
    mdy_df=mdy_df.astype(int)
    
    mdy_df['year']=mdy_df['year'].apply(lambda x: x+1900 if x<100 else x)
    mdy_sorted_df=mdy_df.sort_values(['year','month','day'])
    order = pd.Series(mdy_sorted_df.index)
    
    return order

date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64